## 2010-2019 kospi200 편입여부 확인

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
df = pd.read_csv('./data/crawling/preprocessed/Lv.1_dongchan.csv',encoding='euc-kr',index_col=0)
df = df.drop(['str_tokens','url'],axis=1)
df_boolean = pd.read_csv('./data/kospi/raw/KOSPI200_편입여부1.csv',low_memory=False)

In [3]:
def length_adjust(str_) : 
    return (2-len(str_)) * '0' + str_

def date_convertor(date) : 
    year = length_adjust(str(date.year))
    month = length_adjust(str(date.month))
    day = length_adjust(str(date.day))
    yyyymmdd = int(year+month+day)
    return yyyymmdd

In [4]:
dts = df_boolean.iloc[1:,0] #datetime형식을 통일시켜준다
dates_str = []
for idx,i in enumerate(dts):
    i = df_boolean.iloc[idx+1,0][0:4]+df_boolean.iloc[idx+1,0][5:7]+df_boolean.iloc[idx+1,0][8:10] 
    dates_str.append(i)

dates_str.insert(0,'Symbol Name')


symbol_ls = [x for x in df_boolean.columns][1:]
crp_ls = []
for i in symbol_ls:
    if df_boolean[i][1:].isin(['Y']).all() == True:   #전체 kospi기업에서 10년동안 kospi200에 편입되었던 기업들만 추출
        crp_ls.append(i)


dts = df_boolean.iloc[1:,0] #datetime형식을 통일시켜준다
dates_str = []
for idx,i in enumerate(dts):
    i = df_boolean.iloc[idx+1,0][0:4]+df_boolean.iloc[idx+1,0][5:7]+df_boolean.iloc[idx+1,0][8:10] 
    dates_str.append(i)

dates_str.insert(0,'Symbol Name')

df_boolean = df_boolean.loc[:,crp_ls]
df_boolean.index = dates_str


In [5]:
df_boolean.head()

,A000070,A000100,A000120,A000150,A000210,A000240,A000270,A000640,A000660,A000670,...,A069960,A071050,A078930,A086790,A090430,A093050,A093370,A096770,A097950,A105560
Symbol Name,삼양홀딩스,유한양행,CJ대한통운,두산,대림산업,한국테크놀로지그룹,기아차,동아쏘시오홀딩스,SK하이닉스,영풍,...,현대백화점,한국금융지주,GS,하나금융지주,아모레퍼시픽,LF,후성,SK이노베이션,CJ제일제당,KB금융
20100101,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y
20100102,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y
20100103,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y
20100104,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y


## kospi200 기업의 일별 종가 추출

FinanceDataReader 패키지를 이용하여 2010년부터 2019년 말까지 해당 기업의 종가를 가져오는 함수를 구현한다. 개별기업의 주가를 dataframe형식으로 반환하므로 10년동안 지수에 편입되었던 113개의 기업들에 대하여 수정종가, 수익률 , 그리고 naive하게 market return을 제외한 수익률을 구하였다.

In [6]:
def close_price(code): #개별기업코드 입력시 종가 반환
    
    import FinanceDataReader as fdr
    
    df = fdr.DataReader(code, start = '2010-01-01' , end = '2019-12-31')
    df = df['Close']
    
    return df

In [7]:
def get_return(code): #개별기업코드 입력시 수익률 반환
    
    import FinanceDataReader as fdr
    
    df = fdr.DataReader(code, start = '2010-01-01' , end = '2019-12-31')
    df = df['Change']
    
    return df

In [8]:
df_boolean

,A000070,A000100,A000120,A000150,A000210,A000240,A000270,A000640,A000660,A000670,...,A069960,A071050,A078930,A086790,A090430,A093050,A093370,A096770,A097950,A105560
Symbol Name,삼양홀딩스,유한양행,CJ대한통운,두산,대림산업,한국테크놀로지그룹,기아차,동아쏘시오홀딩스,SK하이닉스,영풍,...,현대백화점,한국금융지주,GS,하나금융지주,아모레퍼시픽,LF,후성,SK이노베이션,CJ제일제당,KB금융
20100101,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y
20100102,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y
20100103,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y
20100104,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20191227,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y
20191228,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y
20191229,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y
20191230,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y


In [9]:
symbols = [x for x in df_boolean.iloc[0,:]] #기업명 리스트
t_ls = [x[1:] for x in df_boolean.columns] #종목코드
symbol_dic = {name:value for name,value in zip(symbols,t_ls)}


close_ls = [close_price(x) for x in t_ls] 
return_ls = [get_return(x) for x in t_ls]

df__price = pd.concat([i for i in close_ls],ignore_index=False, axis =1)
df__price.columns = symbols
# df__price = df__price.fillna(int(0)) #str to int.. NaN값은0으로 변환

df_return = pd.concat([i for i in return_ls],ignore_index=False, axis =1)
df_return.columns = symbols
# df_return = df_return.fillna(int(0)) #str to int.. NaN값은0으로 변환


#datetime string으로 변환
dates = df__price.index
dates_ls = list(map(date_convertor,dates)) #datetime string으로 변환
df_price = df__price.copy()

df_price.index = dates_ls
df_return.index = dates_ls

In [10]:
df_return.shape
nan_mean  = np.nanmean(df_return, axis=1) #nan값빼고 market return을 naive하게 계산
demean = df_return.values-nan_mean.reshape(len(nan_mean),1) #market return 제외

df_demean= pd.DataFrame(demean,index=df_return.index, columns = df_return.columns)

In [11]:
df_price.head()

,삼양홀딩스,유한양행,CJ대한통운,두산,대림산업,한국테크놀로지그룹,기아차,동아쏘시오홀딩스,SK하이닉스,영풍,...,현대백화점,한국금융지주,GS,하나금융지주,아모레퍼시픽,LF,후성,SK이노베이션,CJ제일제당,KB금융
20100104,46517,141333,54500,74680,85300,12405,20800,97972,24100,569000,...,113000,34250,33750,33350,94224,31900,3726,117000,207000,59400
20100105,46257,142136,54400,74519,84500,11860,19650,94669,23350,550000,...,108000,34600,33900,33900,91524,30650,3702,121500,220000,58700
20100106,45894,143741,56400,78619,86200,11786,19850,92101,24550,537000,...,105500,34300,34250,33900,86022,30400,3755,122000,220500,58500
20100107,45167,142536,56600,80790,86000,11488,18950,92835,24400,535000,...,106000,33600,34600,33450,84122,30150,3785,121500,218000,57800
20100108,45063,141333,57600,86015,86900,11588,19350,92101,24650,533000,...,103500,35850,34450,33550,82821,30450,3804,122500,218000,57900


In [12]:
df_return.head()

,삼양홀딩스,유한양행,CJ대한통운,두산,대림산업,한국테크놀로지그룹,기아차,동아쏘시오홀딩스,SK하이닉스,영풍,...,현대백화점,한국금융지주,GS,하나금융지주,아모레퍼시픽,LF,후성,SK이노베이션,CJ제일제당,KB금융
20100104,0.004492,-0.011229,-0.038801,-0.026209,0.025240,-0.017581,0.037406,0.038918,0.041037,0.000000,...,0.004444,0.022388,-0.001479,0.013678,0.008563,0.011094,-0.006665,-0.004255,0.007299,-0.005025
20100105,-0.005589,0.005682,-0.001835,-0.002156,-0.009379,-0.043934,-0.055288,-0.033714,-0.031120,-0.033392,...,-0.044248,0.010219,0.004444,0.016492,-0.028655,-0.039185,-0.006441,0.038462,0.062802,-0.011785
20100106,-0.007847,0.011292,0.036765,0.055020,0.020118,-0.006239,0.010178,-0.027126,0.051392,-0.023636,...,-0.023148,-0.008671,0.010324,0.000000,-0.060115,-0.008157,0.014317,0.004115,0.002273,-0.003407
20100107,-0.015841,-0.008383,0.003546,0.027614,-0.002320,-0.025284,-0.045340,0.007970,-0.006110,-0.003724,...,0.004739,-0.020408,0.010219,-0.013274,-0.022087,-0.008224,0.007989,-0.004098,-0.011338,-0.011966
20100108,-0.002303,-0.008440,0.017668,0.064674,0.010465,0.008705,0.021108,-0.007907,0.010246,-0.003738,...,-0.023585,0.066964,-0.004335,0.002990,-0.015466,0.009950,0.005020,0.008230,0.000000,0.001730


In [13]:
df_demean.head()

,삼양홀딩스,유한양행,CJ대한통운,두산,대림산업,한국테크놀로지그룹,기아차,동아쏘시오홀딩스,SK하이닉스,영풍,...,현대백화점,한국금융지주,GS,하나금융지주,아모레퍼시픽,LF,후성,SK이노베이션,CJ제일제당,KB금융
20100104,-0.001326,-0.017046,-0.044618,-0.032027,0.019423,-0.023399,0.031589,0.033100,0.035219,-0.005818,...,-0.001373,0.016570,-0.007297,0.007860,0.002745,0.005276,-0.012483,-0.010073,0.001482,-0.010843
20100105,-0.000428,0.010843,0.003326,0.003005,-0.004218,-0.038773,-0.050127,-0.028553,-0.025959,-0.028231,...,-0.039087,0.015380,0.009605,0.021653,-0.023494,-0.034024,-0.001280,0.043623,0.067963,-0.006624
20100106,-0.013265,0.005874,0.031347,0.049602,0.014701,-0.011657,0.004760,-0.032544,0.045974,-0.029054,...,-0.028566,-0.014088,0.004907,-0.005418,-0.065533,-0.013574,0.008899,-0.001302,-0.003145,-0.008825
20100107,-0.009135,-0.001677,0.010252,0.034320,0.004386,-0.018578,-0.038634,0.014675,0.000596,0.002982,...,0.011445,-0.013702,0.016925,-0.006568,-0.015381,-0.001518,0.014695,0.002608,-0.004632,-0.005260
20100108,-0.009778,-0.015915,0.010192,0.057198,0.002990,0.001229,0.013633,-0.015382,0.002770,-0.011214,...,-0.031060,0.059489,-0.011811,-0.004486,-0.022941,0.002475,-0.002456,0.000755,-0.007476,-0.005745


## crawling data 전처리

In [14]:
def preprocessing(document): #특수문자 기본 전처리 함수
    docu = []
    for doc in document:
        doc = re.sub('[^가-힣0-9\s.]','',doc)
        doc = re.sub('[-=+,#/\?:^$@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',doc)
#         doc = re.sub('[0-9]+','num',doc) #숫자를 special token num으로 치환
        pattern = re.compile(r'\s+') #중복띄어쓰기 제거
        doc = re.sub(pattern,' ',doc)
        doc = doc.replace('\xa0','') 
        doc = doc.replace('\n','')
        docu.append(doc)

    return docu

In [15]:
nan_idx = [x for x in df[df.str.isnull() ==True].index]
df = df.drop(nan_idx,axis=0) #str 결측치 삭제
df.rmk = df.rmk.fillna('nan')
df = df.reset_index(drop=True)

str_ls = [x for x in df['str']]
corrected_ls = preprocessing(str_ls)
df['str'] = corrected_ls

크롤링한 LV1 data(kospi200 한 번이라도 편입되었던 309개 기업의 텍스트)에 대한 토큰화 이전 기본 전처리를 하고 결측치 처리해주었다.

종목코드는 같지만 거래소에 공시되어 있는 기업의 이름과 dart에서 크롤링한 기업의 이름이 상충되는 것이 있어서 이를 거래소 이름으로 통일하여 주었다. 여기서 지수 편입 퇴출 효과를 제거하기 위해 10년동안 한번도 퇴출된 적이 없었던 113개의 기업들로 데이터를 다시 구성하였다.

In [16]:
change_dic = {'케이씨씨':'KCC',
 '케이티':'KT',
'케이티앤지':'KT&G',
'포스코':'POSCO',
'금호석유화학':'금호석유',
 '기아자동차':'기아차',
'롯데칠성음료':'롯데칠성',
 '삼성화재해상보험':'삼성화재',
 '쌍용양회공업': '쌍용양회',
'아모레퍼시픽그룹':'아모레G',
 '한국단자공업':'한국단자',
'한국전력공사':'한국전력',
'현대엘리베이터':'현대엘리베이',
 '현대자동차':'현대차'}


new_crp_ls = []
for i in df.crp_nm:
    for x in change_dic.keys():
        if i == x:
            i = change_dic[x]
    new_crp_ls.append(i)

df.crp_nm = new_crp_ls

symbols = df_boolean.iloc[0,:] 
con_ls = [df[df.crp_nm == i] for i in symbols]
df_all = pd.concat(x for x in con_ls).reset_index(drop=True) #10년동안 편입되어있던 113개의 기업

In [19]:
df_all.head(5)

,crp_nm,crp_cd,연도,rpt_nm,rcp_no,rcp_dt,rmk,str
0,삼양홀딩스,70,2018,사업보고서 (2018.12),20190401004413,20190401,정연,. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 본 경영진단의견서...
1,삼양홀딩스,70,2017,사업보고서 (2017.12),20180402004217,20180402,연,. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항당사가 본 경영진단의견서...
2,삼양홀딩스,70,2016,사업보고서 (2016.12),20170331003683,20170331,연,. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항 당사가 본 경영진단의견...
3,삼양홀딩스,70,2015,사업보고서 (2015.12),20160330002684,20160330,정연,. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항 당사가 본 경영진단의견...
4,삼양홀딩스,70,2014,사업보고서 (2014.12),20150331002399,20150331,연,. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항 당사가 본 경영진단의견...


In [20]:
df_all.str[5]

'. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항 당사가 본 경영진단의견서에서 미래에 발생할 것으로 예측하여 기재 또는 표시한 내용은 동 의견서 작성시점의 사건 및 재무성과에 대한 회사의 의견을 반영한 것입니다. 동 예측정보는 미래의 경영환경과 관련된 다양한 가정에 기초하고 있으며 이러한 가정들은 동 의견서 작성시점의 합리적인 근거에 기초하였지만 외부 환경 회사 내부경영과 관련된 요인 및 기타 변수로 인하여 결과적으로 부정확한 것으로 판명될 수도 있습니다. 당사는 동 의견서의 예측정보 작성시점 이후에 발생하는 위험 또는 불확실성을 반영하기 위하여 예측정보에 기재한 사항을 수정하는 정정보고서를 공시할 의무는 없습니다. 결론적으로 이러한 주의문구는 모든 예측정보에 대하여 적용되며 동 의견서상에서 회사가 예상한 결과 또는 사항이 실현되거나 회사가 당초에 예상한 영향이 발생한다는 확신을 제공할 수 없습니다. 동 의견서에 기재된 예측정보는 동 의견서 작성시점을 기준으로 작성한 것이며 회사가 이러한 위험요인이나 예측정보를 업데이트할 책임이 없음을 유의하시기 바랍니다. 2. 개요 내수경기 침체와 글로벌 석유화학 업계 불황으로 그룹의 화학부문은 부진한 성과를 거두었습니다. 하지만 식품사업 부문은 원료가격 안정과 판매 경쟁력 확보로 비교적 양호한 실적을 거두었으며 의료용구 부문은 치열한 가격경쟁에도 불구하고 매출목표를 달성하였습니다. 당사 및 연결대상 종속회사이하 연결실체와 주요 자회사가 영위하는 주요 사업 부문별 영업현황은 다음과 같습니다.가. 식품부문 식품사업 부문은 유통환경 변화에 선제적으로 대응하기 위해 서브큐 브랜드를 론칭하고 식자재 유통시장에 본격 진출하였습니다. 신뢰할 수 있는 상품과 고객지원 솔루션 제공 체계적인 위생관리와 물류시스템 구축에 힘써 식품사업의 신성장 동력을 확보하겠습니다. 소비재사업 부문은 홈메이드 시장의 리더십을 바탕으로 체중조절식 시장을 새롭게 창출하고 간편식과 숙취해소제 시장에도 적극 진출하는 등 다양한 신제품을 출시하였습니다. 